## Extracción de información de la página del Rues con autenticación

<p><b>TIP:</b> Para descomentar una línea de código en Visual Studio Code, puede señalar la línea que se quiera descomentar y utilizar el comando <b>'Ctrl + k + u'</b>. Por el contrario, para comentar una línea puede usar el comando <b>'Ctrl + k + c'</b>. En caso de ya tenerlos instalados no se debe hacer nada.</p><p><b>NOTA:</b> Se debe descargar <b>Chrome Driver</b> que es el navegador de prueba que permitirá ejecutar la automatización, se recomienda instalarlo en el escritorio ya que el código está para que lo ejecute ahí. Puede descargar Chrome Driver en el siguiente link: <a href='URL'>https://developer.chrome.com/docs/chromedriver/downloads?hl=es-419</a>
</p>

<p>Librerías a cargar, debemos cargar <b>Selenium</b> (automatización de procesos) y <b>Beautifulsoup</b> (web scrapping).</p> Se puede instalar seleccionando todo el bloque de abajo, oprimiendo el comando ctrl+k+u para descomentarlo y luego ejecutandolo

In [ ]:
##%pip install selenium
##%pip install Beautifulsoup 
##%pip install pandas
##%pip install openpyxl

Cargar las librerías necesarias

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
from pathlib import Path

<p><b>Ingresar los NITs a extraer información</b>. Si los tiene en un archivo de excel se pueden cargar con lel siguiente bloque de código abajo, solo debe especificar la ruta donde está el archivo y luego especificar el nombre de la columna donde están los nits en el archivo.</p>

In [ ]:
# nits = [] # Aquí se deben cargar los NITs a los cuales se les quiera extraer la información. 

# Imprimir la lista
print(len(nits))

<p>Si los nits que quiere buscar la información están en un archivo excel:</p>
<p><b>Nota:</b> solo debe reemplazar la ruta del archivo (la cual la puede encontrar dandole click derecho en el archivo y luego en propiedades)</p>

In [ ]:
## Si tiene un archivo excel:
# archivo_nits = pd.read_excel(ruta_del_archivo, dtype=object)
# nits = archivo_nits['nombre_de_la_columna_donde estan_los_nits'].tolist()
# print(len(nits))
# print(nits)

**Iniciar sesión**. Se debe ingresar el usuario y contraseña

In [1]:
# Credenciales de login
username = 'XXXXXX'
password = 'XXXXXX'

Abrir el chromeDriver. Aquí se debe actualizar la ruta donde se haya instalado el ChromeDriver. Por recomendación, lo mejor es instalarlo en la aplicación de escritorio (el usuario se llama practeec, en caso de que ya no sea así debe cambiarlo). Además iniciaremos sesión automaticamente, aunque también se puede hacer de forma manual (si el usuario y/o contraseña cambia, deben cambairse estos valores en el código)

In [ ]:
# Ruta al ejecutable de ChromeDriver (ajusta esta ruta según donde hayas descargado el driver)
chrome_driver_path = Path.home() / 'Desktop' / 'Chromedriver.exe'
# Crear una instancia del servicio de ChromeDriver
service = Service(chrome_driver_path)
# Inicializar el navegador usando el servicio
driver = webdriver.Chrome(service=service)
# URL de la página de login
login_url = "https://www.rues.org.co/?old=true"

# Navegar a la página de login
driver.get(login_url)

# Botón cerrar emergente
xpath_boton_cerrar_emergente = "//button[@class='btn btn-success btn-sm']"
boton_cerrar_emergente = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath_boton_cerrar_emergente)))
driver.execute_script("arguments[0].scrollIntoView();", boton_cerrar_emergente)
boton_cerrar_emergente.click()
# Darle tiempo (5 segundos)
time.sleep(5)

#Hacer click en el botón de acceso
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'login-call'))).click()

# Encontrar los campos de login y enviar las credenciales
try:
    email_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'Email')))
    password_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'Password')))
    
    # Desplazarse a los elementos para asegurarse de que están visibles
    driver.execute_script("arguments[0].scrollIntoView();", email_element)
    driver.execute_script("arguments[0].scrollIntoView();", password_element)
    
    email_element.send_keys(username)
    password_element.send_keys(password)
except (TimeoutException, ElementNotInteractableException) as e:
    print("No se pudo interactuar con los campos de login:", e)
    driver.quit()
    raise

# Encontrar el botón por su texto y hacer clic
driver.find_element(By.CLASS_NAME, 'btn-blue').click() # Ajusta según el botón de submit

### Aquí empieza el ciclo for para los n NITs

In [ ]:
# Matriz a rellenar más adelante
data = {
    'Nit': None,
    'Activo Total': None,
    'Pasivo Total': None,
    'Patrimonio': None,
    'Ingresos Actividad Ordinaria': None,
    'Utilidad/Perdida Operacional': None,
    'Resultado del Periodo': None
}

# Ciclo for para los n NITs enlistados
results = []

for nit in nits:
   
    try:
        # Esperar hasta que el campo de "Número de identificación" esté presente y realizar la consulta
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'txtNIT')))
        driver.find_element(By.ID, 'txtNIT').send_keys(nit)
        driver.find_element(By.ID, "btnConsultaNIT").click()

        # Esperar hasta que el botón esté presente y hacer clic
        
        # Encontrar la celda que contiene el texto "ACTIVA"
        activa_td_xpath = "//table[@id='rmTable2']//tbody//tr//td[text()='ACTIVA']"
        activa_td = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, activa_td_xpath))
        )
        # Encontrar la fila padre de la celda encontrada
        parent_row = activa_td.find_element(By.XPATH, "./ancestor::tr")
        # Encontrar la celda con tabindex="0" en la misma fila
        target_td = parent_row.find_element(By.XPATH, ".//td[@tabindex='0']")
        # Esperar a que la celda con tabindex="0" sea clicable
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, ".//td[@tabindex='0']"))
        )
        # Hacer clic en la celda con tabindex="0"
        target_td.click()

        # Dar click en el link de info
        encontrar_link = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.LINK_TEXT, "nfo")))
        encontrar_link.click()

        # Dar click en el botón de info financiera
        xpath_ver_info_financiera = "(//span[contains(text(), 'Ver información')])[1]"
        ver_info_link = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, xpath_ver_info_financiera)))
        WebDriverWait(driver, 20).until(EC.visibility_of(ver_info_link))
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, xpath_ver_info_financiera)))
        driver.execute_script("arguments[0].scrollIntoView(true);", ver_info_link)
        driver.execute_script("arguments[0].click();", ver_info_link)

        # Dar click en el botón 2024
        xpath_2024 = "//div[@id='2024']//button"
        ver_2024_link = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, xpath_2024)))
        WebDriverWait(driver, 20).until(EC.visibility_of(ver_2024_link))
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath_2024)))
        driver.execute_script("arguments[0].scrollIntoView(true);", ver_2024_link)
        driver.execute_script("arguments[0].click();", ver_2024_link)

        # Esperar hasta que la tabla esté presente
        xpath_tabla_2024 = "//div[@id='collapse-2024']//table"
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath_tabla_2024)))

        # Extraer el contenido de la página
        page_source = driver.page_source

        # Analizar el contenido con BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')

        # Buscar la tabla con datos
        table = soup.select_one('div#collapse-2024 table')
        data = {'NIT': nit}
        if table:
            # Extraer todas las filas de la tabla
            rows = table.find_all('tr')
            for row in rows:
                # Extraer todas las celdas de la fila
                cells = row.find_all('td')
                cell_data = [cell.text.strip() for cell in cells]
                if 'Activo Total' in cell_data[0]:
                    data['Activo Total'] = cell_data[1]
                elif 'Pasivo Total' in cell_data[0]:
                    data['Pasivo Total'] = cell_data[1]
                elif 'Patrimonio' in cell_data[0]:
                    data['Patrimonio'] = cell_data[1]
                elif 'Ingresos Actividad Ordinaria' in cell_data[0]:
                    data['Ingresos Actividad Ordinaria'] = cell_data[1]
                elif 'Utilidad/Perdida Operacional' in cell_data[0]:
                    data['Utilidad/Perdida Operacional'] = cell_data[1]
                elif 'Resultado del Periodo' in cell_data[0]:
                    data['Resultado del Periodo'] = cell_data[1]
        else:
            print("No se encontró la tabla de datos")

        # Agregar los datos del NIT actual a la lista de resultados
        results.append(data)

        # Dar click en el botón de regresar para que se pueda iniciar la otra consulta
        boton_regresar_xpath = "//a[@class='btn-gt']"
        ver_boton_regresar = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, boton_regresar_xpath)))
        WebDriverWait(driver, 20).until(EC.visibility_of(ver_boton_regresar))
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, boton_regresar_xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", ver_boton_regresar)
        driver.execute_script("arguments[0].click();", ver_boton_regresar)

    except TimeoutException as e:
        print(f"Hubo un problema al intentar interactuar con el NIT {nit}:", e)

# Convertir los resultados a un DataFrame de pandas para facilitar la manipulación
df = pd.DataFrame(results)

Hasta aquí iría el ciclo for para cada NIT. Se procede a descargar en formato excel. Por facilidad se descargará en escritorio pero se puede modificar esta ruta si así lo desea

In [ ]:
# Pasar las variables a valor numérico y luego a integer
df = df.apply(pd.to_numeric, errors='coerce')
df = df.fillna(0).astype(int)
print(df.dtypes)

# Ruta carpeta para guardar el archivo
rues_descarga_path = Path.home() / 'Desktop' / '2024_rues.xlsx'
# Crear el archivo excel 
df.to_excel(rues_descarga_path, index=False)

# Cerrar el navegador
driver.quit()